In [ ]:
import os
from pathlib import Path
REPO_ROOT = Path('/home/phil/work/sparkle_motion')
os.chdir(REPO_ROOT)
os.environ["RUN_ID"] = "run_c302c0ce1b30"
os.environ["FINAL_VIDEO_DIR"] = str(REPO_ROOT / "tmp" / "final_videos")
Path(os.environ["FINAL_VIDEO_DIR"]).mkdir(parents=True, exist_ok=True)
print("CWD:", Path.cwd())
print("RUN_ID set to", os.environ["RUN_ID"])
print("FINAL_VIDEO_DIR set to", os.environ["FINAL_VIDEO_DIR"])

# sparkle_motion — Colab A100 runbook
This notebook **only** supports the Google Colab A100 runtime. Every cell assumes you are in `/content` with a GPU attached.
- Step through the cells in order; do not skip the install step.
- Choose whether to work out of Google Drive (persistent) or `/content` (ephemeral) by toggling the workspace config cell.
- The smoke-test cell is a safe stub that checks imports and shows how to run the orchestrator in simulation mode once the stack is installed.

## Notes and expectations
- Always request an A100 runtime in Colab (`Runtime → Change runtime type → GPU → A100`).
- Keep the notebook tab in focus while installs run; cancel/restart the runtime if `pip install` fails.
- The only decision point is whether to mount Google Drive (recommended) or stay in `/content` for a throwaway run.

## Pull the repository into this runtime
Use the next cell when you open this notebook directly from GitHub in Colab. It clones (or updates) the `sparkle_motion` repo inside `/content` so every other helper has access to the full source tree.

In [ ]:
# Cell -1: Clone or update the sparkle_motion repo (Colab-friendly)
from __future__ import annotations

import os
import subprocess
from pathlib import Path
from typing import Optional

REPO_URL = "https://github.com/ekkus93/sparkle_motion.git"
TARGET_DIR = Path("/content/sparkle_motion").resolve()


def _existing_repo_root(start: Path) -> Optional[Path]:
    for candidate in [start, *start.parents]:
        if (candidate / "pyproject.toml").exists():
            return candidate
    return None


def _run_git(*args: str) -> None:
    subprocess.check_call(["git", *args])


current_dir = Path.cwd().resolve()
repo_root = _existing_repo_root(current_dir)

if repo_root:
    print(f"Found existing sparkle_motion repo at {repo_root}")
    os.chdir(repo_root)
else:
    if TARGET_DIR.exists():
        if (TARGET_DIR / ".git").exists():
            print(f"Repository already present at {TARGET_DIR}; pulling latest changes...")
            _run_git("-C", str(TARGET_DIR), "fetch", "--all", "--prune")
            _run_git("-C", str(TARGET_DIR), "pull", "--ff-only")
        else:
            raise RuntimeError(
                f"{TARGET_DIR} exists but is not a git repository. Delete it or update TARGET_DIR."
            )
    else:
        print(f"Cloning {REPO_URL} into {TARGET_DIR} ...")
        _run_git("clone", REPO_URL, str(TARGET_DIR))
    os.chdir(TARGET_DIR)
    repo_root = TARGET_DIR

REPO_ROOT = Path.cwd().resolve()
globals()["REPO_ROOT"] = REPO_ROOT
os.environ["SPARKLE_MOTION_REPO_ROOT"] = str(REPO_ROOT)
print(f"Working directory switched to {REPO_ROOT}")

## Install required ML stack
Run this immediately after cloning. It installs every dependency listed in `requirements-ml.txt` and is required for all later cells.

In [ ]:
# Cell 0: Install everything from requirements-ml.txt (mandatory)
import importlib.util
import os
import subprocess
import sys
from pathlib import Path

if importlib.util.find_spec("google.colab") is None:
    raise RuntimeError("This notebook only runs on Google Colab with an A100 GPU attached.")

REPO_ROOT = Path(os.environ.get("SPARKLE_MOTION_REPO_ROOT", Path.cwd())).resolve()
globals()["REPO_ROOT"] = REPO_ROOT
req_path = REPO_ROOT / "requirements-ml.txt"
if not req_path.exists():
    raise FileNotFoundError(f"requirements-ml.txt not found at {req_path}")

print("Installing Python dependencies (this may take several minutes)...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", str(req_path)])
print("Dependency install complete.")

## Configure workspace inputs
Set the flags below **once**. `USE_DRIVE = True` keeps your assets in Google Drive; setting it to `False` keeps everything under `/content` for a throwaway run. Provide one or more repo IDs via `HF_MODELS`; set `DRY_RUN = True` or leave the list empty to skip downloads.

In [ ]:
# Cell 1: Workspace configuration (choose Drive vs /content)
from pathlib import Path
import os

USE_DRIVE = True                         # True = mount Google Drive, False = keep everything under /content
WORKSPACE_NAME = "SparkleMotion"          # Folder created under MyDrive/ when USE_DRIVE is True
HF_MODELS = [
    "stabilityai/stable-diffusion-xl-base-1.0",
 ]
DRY_RUN = False
MOUNT_POINT = Path("/content/drive")
DRIVE_ROOT = MOUNT_POINT / "MyDrive" / WORKSPACE_NAME
LOCAL_ROOT = Path("/content") / WORKSPACE_NAME

REPO_ROOT = Path(os.environ.get("SPARKLE_MOTION_REPO_ROOT", Path.cwd())).resolve()
globals()["REPO_ROOT"] = REPO_ROOT

workspace_root = DRIVE_ROOT if USE_DRIVE else LOCAL_ROOT
print(f"Configured workspace '{WORKSPACE_NAME}' → {workspace_root}")
print(f"Google Drive enabled: {USE_DRIVE}")
print(f"Models to manage: {HF_MODELS or '[none specified]'}")

## Load secrets from `.env`
Keep a `.env` file in your Drive workspace (or `/content`) with the required values **before** running the next cell. The filesystem ArtifactService ships with this repo, so no external cloud credential setup is required—everything stays on the Colab VM or your mounted Drive.
- `ADK_PROJECT` — the ADK project slug (usually `sparkle-motion`).
- `ADK_API_KEY` — API credential for the ADK control plane. Follow the steps below to create it if you do not already have one.
- `ADK_USE_FIXTURE` — set to `0` to talk to the **real ADK control plane** (the hosted service your team operates). Set to `1` for the **local fixture mode**, which uses canned responses for dry runs and never leaves Colab.
- `ADK_PUBLISH_INTEGRATION` — leave unset unless you plan to run the ADK publish integration tests (set it to `1` only when running those pytest cases so they get collected).

Optional (set them when you know you need the behavior):
- `PRODUCTION_AGENT_BASE` / `SCRIPT_AGENT_BASE` — override the default `http://127.0.0.1:{8200,8101}` endpoints if your agents run elsewhere.
- `FINAL_VIDEO_DIR` — explicit folder for final MP4 downloads; defaults to the workspace’s `final_videos/`.
- GPU toggles such as `SMOKE_IMAGES`, `SMOKE_VIDEOS`, `IMAGES_SDXL_FIXTURE_ONLY`, `VIDEOS_WAN_FIXTURE_ONLY` when you want to exercise real adapters instead of fixtures (set them to `0` to force the real pipelines).

**How to create an `ADK_API_KEY` (all inside this notebook workflow):**
1. Make sure you can reach the ADK control plane for the `sparkle-motion` project. If you have not logged in yet, run `adk auth login --project sparkle-motion` (from Cloud Shell or your laptop) and finish the browser login it launches, or open the ADK console URL you deployed earlier and select the project there.
2. Sign in via either path: (a) open the ADK console URL in a browser and select the `sparkle-motion` project, or (b) run the CLI command above and finish the browser login it launches.
3. Create a key:
   - Console: go to **Projects → sparkle-motion → Security → API keys → Create key**, give it a name such as "colab-notebook", and check the scopes `artifacts.read` + `artifacts.write`.
   - CLI: run `adk keys create --project sparkle-motion --display-name colab-notebook --scopes artifacts.read artifacts.write`.
4. Copy the `apiKey` value (it starts with `sk-`) the moment it is shown and paste it into your `.env` in this Colab workspace:
   ```
   ADK_PROJECT=sparkle-motion
   ADK_API_KEY=sk-live-xxxxxxxx
   ```
   Keep this file out of version control—treat the key like any other secret.
5. Rerun the next cell so `python-dotenv` loads the updated `.env` into the Colab kernel.
6. Whenever you rotate the value, overwrite the entry in `.env` and rerun the loader cell; no other files need to change.

Once the file is populated, run the next cell; it installs `python-dotenv` if needed and loads every variable into the current Colab kernel.


In [ ]:
# Cell 1b: Load secrets from .env using python-dotenv
import importlib.util
import subprocess
import sys
from pathlib import Path


def _ensure_python_dotenv_installed() -> None:
    if importlib.util.find_spec("dotenv") is None:
        print("Installing python-dotenv...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])


_ensure_python_dotenv_installed()
from dotenv import load_dotenv  # type: ignore  # imported after ensuring package

workspace_root = DRIVE_ROOT if USE_DRIVE else LOCAL_ROOT
candidate_paths = [
    REPO_ROOT / ".env.local",
    REPO_ROOT / ".env",
    Path("/content/.env"),
    workspace_root / ".env",
]

env_path = next((path for path in candidate_paths if path.exists()), None)
if env_path is None:
    print(
        "No .env file found. Run scripts/bootstrap_adk_projects.sh --profile local-colab "
        "from the repo root, copy the file into your workspace, and re-run this cell.",
    )
else:
    load_dotenv(env_path, override=True)
    print(f"Loaded environment variables from {env_path}")

In [ ]:
# Cell 2: Mount Google Drive when USE_DRIVE is True (Colab-only)
import importlib.util
import os
from pathlib import Path

if importlib.util.find_spec("google.colab") is None:
    raise RuntimeError("This notebook must run inside Google Colab.")

if not USE_DRIVE:
    workspace_root = LOCAL_ROOT
    workspace_root.mkdir(parents=True, exist_ok=True)
    print(f"USE_DRIVE is False — using ephemeral workspace at {workspace_root}")
else:
    from google.colab import drive

    mount_target = Path(MOUNT_POINT)
    mount_target.mkdir(parents=True, exist_ok=True)
    if os.path.ismount(mount_target):
        print(f"Google Drive already mounted at {mount_target}.")
    else:
        print(f"Mounting Google Drive at {mount_target}...")
        drive.mount(str(mount_target), force_remount=False)

    workspace_root = DRIVE_ROOT
    workspace_root.mkdir(parents=True, exist_ok=True)
    print(f"Workspace directory ready at {workspace_root}")

## Colab preflight helper
Use this helper right after the install cell to confirm ADC auth, env vars, Drive mount, and `/ready` endpoints before touching the control panel. The helper wraps `python -m sparkle_motion.notebook_preflight` so you can rerun it any time during the session.

In [ ]:
# Cell 2a: Run the consolidated Colab preflight checks
from pathlib import Path
from sparkle_motion.notebook_preflight import format_report, run_preflight_checks

workspace_root = DRIVE_ROOT if USE_DRIVE else LOCAL_ROOT

preflight_results = run_preflight_checks(
    requirements_path=REPO_ROOT / "requirements-ml.txt",
    mount_point=MOUNT_POINT,
    workspace_dir=workspace_root,
    ready_endpoints=(
        "http://localhost:8101/ready",
        "http://localhost:8200/ready",
    ),
    pip_mode="install",
    require_drive=USE_DRIVE,
    skip_gpu_checks=False,
)

print(format_report(preflight_results))

## Prepare workspace and download models
Use `scripts/colab_drive_setup.py` to create the Drive folders (or `/content` folders when `USE_DRIVE = False`), optionally download Hugging Face weights, and write a smoke artifact in `outputs/colab_smoke.json`.
- Leave `HF_MODELS` empty for a dry run, or list specific repo IDs to download.

In [ ]:
# Cell 3: Prepare workspace directories and download models
import importlib.util
import subprocess
import sys
from pathlib import Path

if importlib.util.find_spec("google.colab") is None:
    raise RuntimeError("Run this helper inside Google Colab.")

helper_path = REPO_ROOT / "scripts" / "colab_drive_setup.py"
if not helper_path.exists():
    raise FileNotFoundError(f"Helper script not found at {helper_path}")

cmd = [sys.executable, str(helper_path), WORKSPACE_NAME]
if USE_DRIVE:
    cmd.extend(["--mount-point", str(MOUNT_POINT)])
else:
    workspace_root = LOCAL_ROOT
    workspace_root.mkdir(parents=True, exist_ok=True)
    cmd.extend(["--local-root", str(workspace_root)])
for repo_id in HF_MODELS:
    cmd.extend(["--model", repo_id])
if DRY_RUN:
    cmd.append("--dry-run")

print("Running helper:", " ".join(map(str, cmd)))
subprocess.check_call(cmd)

In [ ]:
# Cell 3b: Inspect smoke artifact with per-model status
import json
from pathlib import Path

workspace_root = DRIVE_ROOT if USE_DRIVE else LOCAL_ROOT
smoke_path = workspace_root / "outputs" / "colab_smoke.json"
if smoke_path.exists():
    data = json.loads(smoke_path.read_text(encoding="utf-8"))
    status = "OK" if data.get("ok") else "FAILED"
    print(f"Smoke status: {status}")
    for model in data.get("models", []):
        sample = model.get("sample_file") or "n/a"
        print(
            f"- {model['repo_id']}: {model['status']} "
            f"({model.get('files_present', 0)} files, sample={sample})"
)
else:
    print(f"No smoke artifact found at {smoke_path}. Run the helper once to generate it.")

In [ ]:
# Cell 4: Smoke-test stub for the orchestrator (safe, non-destructive)
# This cell attempts to import the orchestrator package and reports what is available.
try:
    import sparkle_motion.orchestrator as orchestrator_mod
    print('Imported sparkle_motion.orchestrator ->', orchestrator_mod)
    if hasattr(orchestrator_mod, 'Runner'):
        print('Runner class is available. Instantiate it for a simulation run:')
        print("  from sparkle_motion.orchestrator import Runner")
        print("  r = Runner(run_dir='runs')")
        print("  # then use r.run(...) or similar per your orchestrator API")
    else:
        print('Runner class not found — inspect src/sparkle_motion/orchestrator.py for usage.')
except Exception as e:
    print('Could not import orchestrator module:', e)
    print('Ensure the package is installed (pip install -e .) and rerun this cell.')

## Workflow Agent server controls
These helpers let you start and stop the local Workflow Agent FastAPI apps (script and production agents) directly from the notebook. They simply launch `uvicorn` in the background, track each process (`pid`), and make it easy to shut them down again when you finish. Run them only when you are *not* already running the servers in another terminal.


In [ ]:
import os
import signal
import subprocess
import sys
from pathlib import Path
from typing import Dict, NamedTuple

try:
    import ipywidgets as widgets
    from IPython.display import display
except ImportError as exc:  # pragma: no cover - notebook utility guard
    raise RuntimeError("ipywidgets is required for the Workflow Agent controls") from exc


class ServerProcess(NamedTuple):
    process: subprocess.Popen
    log_path: Path
    log_handle: object


SERVER_CONFIGS: Dict[str, Dict[str, object]] = {
    "script_agent": {
        "app": "sparkle_motion.function_tools.script_agent.entrypoint:app",
        "port": 8101,
    },
    "production_agent": {
        "app": "sparkle_motion.function_tools.production_agent.entrypoint:app",
        "port": 8200,
    },
}

RUNNING_SERVERS: Dict[str, ServerProcess] = {}

PYTHONPATH = os.pathsep.join(
    filter(None, {str(Path(REPO_ROOT)), str(Path(REPO_ROOT) / "src"), os.environ.get("PYTHONPATH", "")})
)


def _server_cmd(name: str) -> list[str]:
    cfg = SERVER_CONFIGS[name]
    return [
        sys.executable,
        "-m",
        "uvicorn",
        cfg["app"],
        "--host",
        os.environ.get("WORKFLOW_AGENT_HOST", "127.0.0.1"),
        "--port",
        str(cfg["port"]),
        "--no-access-log",
    ]


def start_server(name: str) -> None:
    server = RUNNING_SERVERS.get(name)
    if server and server.process.poll() is None:
        raise RuntimeError(f"{name} already running (pid={server.process.pid})")
    env = os.environ.copy()
    env["PYTHONPATH"] = PYTHONPATH
    log_path = Path(REPO_ROOT) / "tmp" / f"{name}.log"
    log_path.parent.mkdir(parents=True, exist_ok=True)
    log_handle = open(log_path, "ab", buffering=0)
    proc = subprocess.Popen(
        _server_cmd(name),
        env=env,
        stdout=log_handle,
        stderr=subprocess.STDOUT,
    )
    RUNNING_SERVERS[name] = ServerProcess(process=proc, log_path=log_path, log_handle=log_handle)
    print(f"Started {name} on port {SERVER_CONFIGS[name]['port']} (pid={proc.pid}). Logs → {log_path}")


def stop_server(name: str, *, sig: int = signal.SIGTERM) -> None:
    server = RUNNING_SERVERS.get(name)
    if not server:
        print(f"{name} has not been started from this notebook.")
        return
    proc = server.process
    if proc.poll() is not None:
        print(f"{name} already stopped (pid={proc.pid}).")
    else:
        proc.send_signal(sig)
        try:
            proc.wait(timeout=10)
        except subprocess.TimeoutExpired:
            proc.kill()
            proc.wait()
        print(f"Stopped {name} (pid={proc.pid}).")
    server.log_handle.close()
    RUNNING_SERVERS.pop(name, None)


def list_servers() -> Dict[str, str]:
    status = {}
    for name in SERVER_CONFIGS:
        proc = RUNNING_SERVERS.get(name)
        if proc and proc.process.poll() is None:
            status[name] = f"running (pid={proc.process.pid})"
        else:
            status[name] = "stopped"
    return status


server_selector = widgets.Dropdown(options=list(SERVER_CONFIGS.keys()), description="Server")
start_button = widgets.Button(description="Start", button_style="success")
stop_button = widgets.Button(description="Stop", button_style="danger")
status_button = widgets.Button(description="Status", button_style="info")
output = widgets.Output()


def _handle_start(_: widgets.Button) -> None:
    with output:
        output.clear_output()
        try:
            start_server(server_selector.value)
        except Exception as exc:  # pragma: no cover - notebook UX guard
            print(f"Failed to start {server_selector.value}: {exc}")


def _handle_stop(_: widgets.Button) -> None:
    with output:
        output.clear_output()
        stop_server(server_selector.value)


def _handle_status(_: widgets.Button) -> None:
    with output:
        output.clear_output()
        for name, state in list_servers().items():
            print(f"{name}: {state}")


start_button.on_click(_handle_start)
stop_button.on_click(_handle_stop)
status_button.on_click(_handle_status)

controls = widgets.HBox([server_selector, start_button, stop_button, status_button])
display(widgets.VBox([controls, output]))


## Filesystem ArtifactService shim controls
Use these helpers to export the required environment variables, launch the local ArtifactService shim, and verify `/healthz` before switching the notebook to `ARTIFACTS_BACKEND=filesystem`. The buttons below wrap `scripts/filesystem_artifacts.py env|serve|health`, so they stay consistent with the new CLI workflow.

In [ ]:
import os
import signal
import subprocess
import sys
from pathlib import Path
from typing import Dict

try:
    import ipywidgets as widgets
    from IPython.display import display
except ImportError as exc:  # pragma: no cover - notebook helper
    raise RuntimeError("ipywidgets is required for the filesystem shim controls") from exc

REPO_ROOT = Path(os.environ.get("SPARKLE_MOTION_REPO_ROOT", Path.cwd())).resolve()
SRC_PATH = REPO_ROOT / "src"
FS_SHIM_HOST = os.environ.get("FILESYSTEM_SHIM_HOST", "127.0.0.1")
FS_SHIM_PORT = int(os.environ.get("FILESYSTEM_SHIM_PORT", "7077"))
FS_ROOT = (DRIVE_ROOT if USE_DRIVE else LOCAL_ROOT) / "artifacts_fs"
FS_ROOT.mkdir(parents=True, exist_ok=True)
_shim_process: subprocess.Popen | None = None
_shim_log_handle = None
_shim_log_path = REPO_ROOT / "tmp" / "filesystem_shim.log"
_shim_log_path.parent.mkdir(parents=True, exist_ok=True)

def _shim_env() -> Dict[str, str]:
    env = os.environ.copy()
    env.setdefault("ARTIFACTS_BACKEND", "filesystem")
    env.setdefault("ARTIFACTS_FS_ROOT", str(FS_ROOT))
    env.setdefault("ARTIFACTS_FS_INDEX", str(FS_ROOT / "index.db"))
    env.setdefault("ARTIFACTS_FS_BASE_URL", f"http://{FS_SHIM_HOST}:{FS_SHIM_PORT}")
    env.setdefault("ARTIFACTS_FS_TOKEN", env.get("ARTIFACTS_FS_TOKEN") or "local-fs-token")
    env["PYTHONPATH"] = os.pathsep.join(filter(None, {str(REPO_ROOT), str(SRC_PATH), env.get("PYTHONPATH", "")}))
    return env

def _start_filesystem_shim(_: widgets.Button | None = None) -> None:
    global _shim_process, _shim_log_handle
    if _shim_process and _shim_process.poll() is None:
        with shim_output:
            shim_output.clear_output()
            print(f"Filesystem shim already running on {FS_SHIM_HOST}:{FS_SHIM_PORT} (pid={_shim_process.pid}).")
        return
    env = _shim_env()
    _shim_log_handle = open(_shim_log_path, "ab", buffering=0)
    cmd = [
        sys.executable,
        "scripts/filesystem_artifacts.py",
        "serve",
        "--host",
        FS_SHIM_HOST,
        "--port",
        str(FS_SHIM_PORT),
    ]
    _shim_process = subprocess.Popen(cmd, env=env, stdout=_shim_log_handle, stderr=subprocess.STDOUT)
    with shim_output:
        shim_output.clear_output()
        print(f"Started filesystem shim on {FS_SHIM_HOST}:{FS_SHIM_PORT} (pid={_shim_process.pid}).")
        print(f"Logs → {_shim_log_path}")

def _stop_filesystem_shim(_: widgets.Button | None = None) -> None:
    global _shim_process, _shim_log_handle
    if not _shim_process:
        with shim_output:
            shim_output.clear_output()
            print("Filesystem shim has not been started from this notebook.")
        return
    if _shim_process.poll() is None:
        _shim_process.send_signal(signal.SIGTERM)
        try:
            _shim_process.wait(timeout=10)
        except subprocess.TimeoutExpired:
            _shim_process.kill()
            _shim_process.wait()
    if _shim_log_handle:
        _shim_log_handle.close()
        _shim_log_handle = None
    with shim_output:
        shim_output.clear_output()
        print("Filesystem shim stopped.")
    _shim_process = None

def _shim_status(_: widgets.Button | None = None) -> None:
    with shim_output:
        shim_output.clear_output()
        if _shim_process and _shim_process.poll() is None:
            print(f"Running on {FS_SHIM_HOST}:{FS_SHIM_PORT} (pid={_shim_process.pid}).")
        else:
            print("Filesystem shim is stopped.")

def _print_env_exports(_: widgets.Button | None = None) -> None:
    env = _shim_env()
    cmd = [
        sys.executable,
        "scripts/filesystem_artifacts.py",
        "env",
        "--shell",
        "bash",
        "--root",
        env["ARTIFACTS_FS_ROOT"],
        "--index",
        env["ARTIFACTS_FS_INDEX"],
        "--token",
        env["ARTIFACTS_FS_TOKEN"],
        "--emit-token",
    ]
    result = subprocess.run(cmd, capture_output=True, text=True, env=env)
    with env_output:
        env_output.clear_output()
        if result.returncode == 0:
            print(result.stdout.strip())
        else:
            print(result.stderr or result.stdout)

def _health_probe(_: widgets.Button | None = None) -> None:
    env = _shim_env()
    cmd = [
        sys.executable,
        "scripts/filesystem_artifacts.py",
        "health",
        "--url",
        f"http://{FS_SHIM_HOST}:{FS_SHIM_PORT}/healthz",
        "--token",
        env["ARTIFACTS_FS_TOKEN"],
    ]
    result = subprocess.run(cmd, capture_output=True, text=True, env=env)
    with env_output:
        env_output.clear_output()
        if result.returncode == 0:
            print(result.stdout.strip())
        else:
            print(result.stderr or result.stdout)

start_button = widgets.Button(description="Start shim", button_style="success")
stop_button = widgets.Button(description="Stop shim", button_style="danger")
status_button = widgets.Button(description="Status", button_style="info")
env_button = widgets.Button(description="Show env exports")
health_button = widgets.Button(description="Probe /healthz", icon="heartbeat")
shim_output = widgets.Output()
env_output = widgets.Output()

start_button.on_click(_start_filesystem_shim)
stop_button.on_click(_stop_filesystem_shim)
status_button.on_click(_shim_status)
env_button.on_click(_print_env_exports)
health_button.on_click(_health_probe)

controls_row = widgets.HBox([start_button, stop_button, status_button, env_button, health_button])
display(widgets.VBox([controls_row, shim_output, env_output]))

## Quickstart: Launch the control panel
Run this cell after the FunctionTools are live (script_agent + production_agent listening on localhost). It imports `create_control_panel`, instantiates the widgets with the `local-colab` profile, and stores the panel in `control_panel` so later helpers (status polling, final deliverable preview) can reuse the same run metadata.


In [ ]:
from pathlib import Path
import os
import sys

REPO_ROOT = Path(os.environ.get("SPARKLE_MOTION_REPO_ROOT", Path.cwd())).resolve()
globals()["REPO_ROOT"] = REPO_ROOT
if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))
SRC_PATH = REPO_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

In [ ]:
# Quickstart cell: import and display the ipywidgets control panel
from notebooks.control_panel import create_control_panel

print("Launching control panel with endpoints from configs/tool_registry.yaml (profile='local-colab').")
control_panel = create_control_panel()
control_panel

In [ ]:
# Helper: set run_id for artifacts viewing/tests
TARGET_RUN_ID = "run_a12af6a94ab5"  # Latest local run with finalize artifacts
if "control_panel" in globals():
    if hasattr(control_panel, "run_id_input"):
        control_panel.run_id_input.value = TARGET_RUN_ID
    state = getattr(control_panel, "state", None)
    if state is not None:
        state.last_run_id = TARGET_RUN_ID
    print(f"Control panel run_id set to {TARGET_RUN_ID}")
else:
    print("control_panel is not initialized; run the quickstart cell first.")


In [ ]:
# Launch a fresh production run for notebook verification
import json
import time
from pathlib import Path
import httpx

PLAN_PATH = (REPO_ROOT / "artifacts" / "Test_Film.json").resolve()
if not PLAN_PATH.exists():
    raise FileNotFoundError(f"Sample plan not found at {PLAN_PATH}")
plan_payload_raw = json.loads(PLAN_PATH.read_text())
plan_payload = plan_payload_raw.get("validated_plan") or plan_payload_raw
request_body = {"mode": "run", "plan": plan_payload}
with httpx.Client(timeout=60.0) as client:
    invoke_resp = client.post(f"{PRODUCTION_AGENT_BASE}/invoke", json=request_body)
    if invoke_resp.status_code >= 400:
        snippet = invoke_resp.text[:800]
        print("Invoke failed (truncated):", snippet)
        invoke_resp.raise_for_status()
    invoke_data = invoke_resp.json()
run_id_new = invoke_data["run_id"]
print("Production run_id:", run_id_new)

def _poll_status(run_id: str, *, timeout_s: float = 40.0) -> None:
    with httpx.Client(timeout=10.0) as client:
        start = time.time()
        attempt = 0
        while time.time() - start < timeout_s:
            status = client.get(f"{PRODUCTION_AGENT_BASE}/status", params={"run_id": run_id}).json()
            print(
            f"poll {attempt}: status={status.get('status')} steps={len(status.get('steps', []))}"
            )
            if status.get("status") == "succeeded":
                return
            attempt += 1
            time.sleep(0.5)
        raise RuntimeError("Run did not complete within timeout")

_poll_status(run_id_new)
if "control_panel" in globals():
    if hasattr(control_panel, "run_id_input"):
        control_panel.run_id_input.value = run_id_new
    state = getattr(control_panel, "state", None)
    if state is not None:
        state.last_run_id = run_id_new
print("Control panel updated with new run_id.")

## Notebook control panel prototype (advanced)
Need to override the timeout, point at a different profile, or inspect the underlying widgets? Use the cell below to instantiate `ControlPanel` manually. It shows how to swap endpoint profiles, tweak HTTP timeouts, and still reuse the global `control_panel` handle for downstream helpers.


In [ ]:
# Advanced control panel prototype: customize endpoints/timeouts
from notebooks.control_panel import ControlPanel, PanelEndpoints

CUSTOM_PROFILE = "local-colab"  # change to another profile defined in configs/tool_registry.yaml
CUSTOM_TIMEOUT_S = 45.0

print(f"Building ControlPanel(profile={CUSTOM_PROFILE!r}, timeout={CUSTOM_TIMEOUT_S}s)...")
custom_endpoints = PanelEndpoints.from_registry(profile=CUSTOM_PROFILE)
advanced_control_panel = ControlPanel(endpoints=custom_endpoints, http_timeout_s=CUSTOM_TIMEOUT_S)

# Keep downstream helpers working by updating the shared reference.
control_panel = advanced_control_panel
advanced_control_panel.container

## Final deliverable preview & download
Use this helper after a production run completes. It fetches the `video_final` artifact from `finalize`, embeds it inline, and offers a Google Colab download when available.

In [ ]:
# Optional: set FINAL_VIDEO_DIR based on workspace choice
import os
from pathlib import Path

workspace_root = DRIVE_ROOT if USE_DRIVE else LOCAL_ROOT
final_dir = workspace_root / "final_videos"
final_dir.mkdir(parents=True, exist_ok=True)
os.environ["FINAL_VIDEO_DIR"] = str(final_dir)
print(f"FINAL_VIDEO_DIR set to {final_dir}")

In [ ]:
# Cell 5: Final deliverable helper (finalize)
import importlib
import os
import subprocess
from pathlib import Path
from typing import Any, Dict, Optional

import httpx
from IPython.display import HTML, display

from notebooks import preview_helpers

PRODUCTION_AGENT_BASE = os.environ.get("PRODUCTION_AGENT_BASE", "http://127.0.0.1:8200")
FINALIZE_STAGE = "finalize"
DOWNLOAD_DIR = Path(os.environ.get("FINAL_VIDEO_DIR", "/content/final_videos"))
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

colab_files = None
try:
    _colab_spec = importlib.util.find_spec("google.colab.files")
except ModuleNotFoundError:
    _colab_spec = None
if _colab_spec:
    colab_files = importlib.import_module("google.colab.files")


def _current_run_id() -> str:
    if "control_panel" in globals():
        cp = globals()["control_panel"]
        run_value = getattr(getattr(cp, "run_id_input", None), "value", "")
        if run_value and run_value.strip():
            return run_value.strip()
        state = getattr(cp, "state", None)
        if state and getattr(state, "last_run_request_id", None):
            return state.last_run_request_id
    return os.environ.get("RUN_ID", "").strip()


RUN_ID = _current_run_id()
if not RUN_ID:
    raise RuntimeError(
        "Set RUN_ID (or populate control_panel.run_id_input) before running the final deliverable helper.",
    )


def _fetch_stage_manifest(run_id: str) -> Dict[str, Any]:
    stage_manifest = preview_helpers.fetch_stage_manifest(
        base_url=PRODUCTION_AGENT_BASE,
        run_id=run_id,
        stage=FINALIZE_STAGE,
    )
    summary = preview_helpers.render_stage_summary(stage_manifest)
    print(summary)
    finalize_status = stage_manifest.get("status") or stage_manifest.get("state")
    if finalize_status:
        print(f"Finalize status => {finalize_status}")
    return stage_manifest


def _locate_video_final(stage_manifest: Dict[str, Any]) -> Dict[str, Any]:
    for entry in stage_manifest.get("artifacts") or []:
        if entry.get("artifact_type") == "video_final":
            return entry
    raise RuntimeError(
        "No video_final artifact found. Ensure finalize succeeded or resume production_agent with resume_from='finalize'.",
    )


def ensure_local_video(entry: Dict[str, Any], run_id: str) -> Path:
    local_path = entry.get("local_path")
    if local_path:
        candidate = Path(local_path).expanduser()
        if candidate.exists():
            return candidate
    target = DOWNLOAD_DIR / f"{run_id}_video_final.mp4"
    download_url = entry.get("download_url")
    if download_url:
        with httpx.Client(timeout=None) as client:
            with client.stream("GET", download_url) as resp:
                resp.raise_for_status()
                with target.open("wb") as handle:
                    for chunk in resp.iter_bytes():
                        handle.write(chunk)
        return target
    artifact_uri = entry.get("artifact_uri")
    if artifact_uri:
        result = subprocess.run(
            ["adk", "artifacts", "download", artifact_uri, str(target)],
            capture_output=True,
            text=True,
            check=False,
        )
        if result.returncode == 0 and target.exists():
            return target
        print("ADK artifact download failed:")
        print(result.stderr or result.stdout)
    raise RuntimeError(
        "Unable to download the final video locally. Provide download_url or local_path in the artifacts manifest.",
    )


def render_preview_video(preview_entry: Optional[Dict[str, Any]]) -> None:
    if not preview_entry:
        print("No preview clip available from /artifacts preview metadata.")
        return
    local_path = preview_entry.get("local_path")
    if local_path and Path(local_path).exists():
        display(HTML("<strong>Preview clip</strong>"))
        preview_widget = preview_helpers.create_video_widget(
            {"local_path": local_path, "artifact_type": "preview_clip"},
            width=480,
        )
        display(preview_widget)
        return
    download_url = preview_entry.get("download_url")
    if download_url:
        print(f"Preview available remotely (download_url={download_url}).")
    else:
        print("Preview metadata present but no local path or download URL; see artifact_uri for details.")


stage_manifest = _fetch_stage_manifest(RUN_ID)
preview_entry = (stage_manifest.get("preview") or {}).get("video")
final_entry = _locate_video_final(stage_manifest)

render_preview_video(preview_entry)
video_path = ensure_local_video(final_entry, RUN_ID)
final_entry["local_path"] = str(video_path)

info_html = f"""
<p><strong>Run ID:</strong> {RUN_ID}</p>
<p><strong>Artifact URI:</strong> {final_entry.get('artifact_uri', 'n/a')}</p>
<p><strong>Local path:</strong> {video_path}</p>
"""
display(HTML(info_html))

display(preview_helpers.create_video_widget(final_entry, width=640))

if colab_files is not None:
    colab_files.download(str(video_path))
else:
    print("Download helper available only inside Google Colab. Share the path above manually if running elsewhere.")

## Artifacts viewer
Use this helper to inspect `/artifacts` responses without leaving the notebook. It shares the `control_panel` run metadata when available, lets you scope by stage (e.g., `finalize`), and can poll automatically so new artifacts appear as production advances.

In [ ]:
# Cell 4c: Artifacts viewer helper
import asyncio
import json
import os
from typing import Any, Dict

import httpx
import ipywidgets as widgets
from IPython.display import HTML, display

from notebooks import preview_helpers

PRODUCTION_AGENT_BASE = os.environ.get("PRODUCTION_AGENT_BASE", "http://127.0.0.1:8200")
ARTIFACTS_ENDPOINT = f"{PRODUCTION_AGENT_BASE}/artifacts"

if "artifact_viewer_state" in globals():
    existing_task = globals()["artifact_viewer_state"].get("task")
    if existing_task and not existing_task.done():
        existing_task.cancel()

artifact_viewer_state = {"task": None}


def _artifact_viewer_run_id() -> str:
    if "control_panel" in globals():
        cp = globals()["control_panel"]
        run_widget = getattr(cp, "run_id_input", None)
        candidate = getattr(run_widget, "value", "")
        if candidate and candidate.strip():
            return candidate.strip()
        state = getattr(cp, "state", None)
        if state and getattr(state, "last_run_request_id", None):
            return state.last_run_request_id
    return os.environ.get("RUN_ID", "").strip()


def _format_status_html(message: str, *, ok: bool) -> str:
    color = "#3c763d" if ok else "#d9534f"
    return f"<span style='color:{color}; font-size:0.9em;'>{message}</span>"


def _iter_artifacts(payload: Dict[str, Any]):
    stages = payload.get("stages") or []
    for stage in stages:
        for artifact in stage.get("artifacts") or []:
            yield artifact
    for artifact in payload.get("artifacts") or []:
        yield artifact


def _render_artifacts(payload: Dict[str, Any]) -> None:
    artifacts = list(_iter_artifacts(payload))
    stages = payload.get("stages") or []
    with artifacts_output:
        artifacts_output.clear_output()
        print(f"Artifacts returned: {len(artifacts)}")
        if stages:
            for stage in stages:
                stage_label = stage.get("stage") or stage.get("stage_id") or "stage"
                print(f"\nStage: {stage_label}")
                print(preview_helpers.render_stage_summary(stage))
                previewable = [entry for entry in (stage.get("artifacts") or []) if entry.get("local_path")]
                if previewable:
                    preview_helpers.display_artifact_previews(
                        {**stage, "artifacts": previewable},
                        max_items=4,
                        video_width=360,
                    )
                else:
                    print("Local previews unavailable yet; see raw payload below.")
        else:
            print("No stage sections returned; dumping raw payload.")
        print("\nFull payload:\n")
        print(json.dumps(payload, indent=2, ensure_ascii=False))


def _fetch_artifacts_sync(run_id: str, stage: str) -> Dict[str, Any]:
    params = {"run_id": run_id}
    if stage:
        params["stage"] = stage
    with httpx.Client(timeout=30.0) as client:
        resp = client.get(ARTIFACTS_ENDPOINT, params=params)
        resp.raise_for_status()
        data = resp.json()
        if not isinstance(data, dict):
            raise RuntimeError("Unexpected artifacts response payload")
        return data


async def _fetch_artifacts_async(client: httpx.AsyncClient, run_id: str, stage: str) -> Dict[str, Any]:
    params = {"run_id": run_id}
    if stage:
        params["stage"] = stage
    resp = await client.get(ARTIFACTS_ENDPOINT, params=params)
    resp.raise_for_status()
    data = resp.json()
    if not isinstance(data, dict):
        raise RuntimeError("Unexpected artifacts response payload")
    return data


def _stop_artifact_poll(*, from_toggle: bool = False) -> None:
    task = artifact_viewer_state.get("task")
    if task and not task.done():
        task.cancel()
    artifact_viewer_state["task"] = None
    if not from_toggle:
        auto_refresh_toggle.value = False


def _handle_manual_refresh(_: Any) -> None:
    run_id = run_id_input.value.strip() or _artifact_viewer_run_id()
    if not run_id:
        status_label.value = _format_status_html("Set a Run ID to fetch artifacts.", ok=False)
        with artifacts_output:
            artifacts_output.clear_output()
            print("Provide a Run ID before refreshing artifacts.")
        return
    try:
        payload = _fetch_artifacts_sync(run_id, stage_input.value.strip())
    except httpx.HTTPError as exc:
        status_label.value = _format_status_html(f"Fetch failed: {exc}", ok=False)
        return
    except Exception as exc:
        status_label.value = _format_status_html(f"Unexpected error: {exc}", ok=False)
        return
    _render_artifacts(payload)
    status_label.value = _format_status_html("Artifacts refreshed.", ok=True)


def _handle_auto_toggle(change: Dict[str, Any]) -> None:
    if change.get("new"):
        _start_artifact_poll()
    else:
        _stop_artifact_poll(from_toggle=True)


def _start_artifact_poll() -> None:
    run_id = run_id_input.value.strip() or _artifact_viewer_run_id()
    if not run_id:
        status_label.value = _format_status_html("Set a Run ID before enabling auto-refresh.", ok=False)
        auto_refresh_toggle.value = False
        return
    loop = asyncio.get_event_loop()

    async def _poll() -> None:
        try:
            async with httpx.AsyncClient(timeout=30.0) as client:
                while auto_refresh_toggle.value:
                    stage = stage_input.value.strip()
                    active_run_id = run_id_input.value.strip() or _artifact_viewer_run_id()
                    if not active_run_id:
                        status_label.value = _format_status_html("Run ID cleared; stopping auto-refresh.", ok=False)
                        _stop_artifact_poll()
                        return
                    try:
                        payload = await _fetch_artifacts_async(client, active_run_id, stage)
                    except httpx.HTTPError as exc:
                        status_label.value = _format_status_html(f"Auto-refresh failed: {exc}", ok=False)
                        _stop_artifact_poll()
                        return
                    except Exception as exc:
                        status_label.value = _format_status_html(f"Error: {exc}", ok=False)
                        _stop_artifact_poll()
                        return
                    _render_artifacts(payload)
                    status_label.value = _format_status_html("Auto-refresh OK.", ok=True)
                    interval = max(2.0, float(interval_input.value or 4.0))
                    await asyncio.sleep(interval)
        except asyncio.CancelledError:
            status_label.value = _format_status_html("Auto-refresh stopped.", ok=True)

    _stop_artifact_poll(from_toggle=True)
    artifact_viewer_state["task"] = loop.create_task(_poll())


run_id_input = widgets.Text(
    value=_artifact_viewer_run_id(),
    description="Run ID",
    placeholder="production run id",
    layout=widgets.Layout(width="50%"),
)
stage_input = widgets.Text(
    value="finalize",
    description="Stage filter",
    placeholder="finalize (default)",
    tooltip="Set to finalize to inspect final artifacts.",
    layout=widgets.Layout(width="45%"),
)
refresh_button = widgets.Button(description="Refresh", icon="refresh")
auto_refresh_toggle = widgets.ToggleButton(description="Auto-refresh", icon="repeat", value=False)
interval_input = widgets.BoundedFloatText(value=4.0, min=2.0, max=60.0, step=1.0, description="Interval (s)")
status_label = widgets.HTML(value=_format_status_html("Idle", ok=True))
artifacts_output = widgets.Output(
    layout=widgets.Layout(border="1px solid #ddd", min_height="160px", max_height="360px", overflow="auto")
)

refresh_button.on_click(_handle_manual_refresh)
auto_refresh_toggle.observe(_handle_auto_toggle, names="value")

controls_row_1 = widgets.HBox([run_id_input, stage_input])
controls_row_2 = widgets.HBox([refresh_button, auto_refresh_toggle, interval_input, status_label])
artifacts_viewer_panel = widgets.VBox([controls_row_1, controls_row_2, artifacts_output])

display(artifacts_viewer_panel)

In [ ]:
# Sync artifacts viewer Run ID widget with control panel
if "control_panel" in globals() and hasattr(control_panel, "run_id_input"):
    run_id_input.value = control_panel.run_id_input.value
print("Artifacts viewer run_id now:", run_id_input.value)


In [ ]:
# Trigger a manual artifacts refresh for verification logs
_handle_manual_refresh(None)
print("Artifacts viewer manual refresh invoked.")


In [ ]:
# Capture artifacts payload for notebook log (single fetch)
import json
current_run = run_id_input.value.strip() or _artifact_viewer_run_id()
payload_snapshot = _fetch_artifacts_sync(current_run, stage_input.value.strip())
stage_names = []
for stage in payload_snapshot.get("stages", []):
    stage_names.append(stage.get("stage") or stage.get("stage_id"))
print(json.dumps(
    {
        "run_id": current_run,
        "artifact_count": len(list(_iter_artifacts(payload_snapshot))),
        "stages": stage_names,
    },
    indent=2,
    ensure_ascii=False,
))


## Filesystem artifact retention helper
Use this cell to trim `ARTIFACTS_FS_ROOT` when running the filesystem shim. It validates backend settings, shells out to `scripts/filesystem_artifacts.py prune`, streams logs inline, and defaults to dry-run mode so you can inspect the plan before deleting artifacts.

In [ ]:
import os
import shlex
import subprocess
import sys
from pathlib import Path
from typing import List

import ipywidgets as widgets

REPO_ROOT = Path.cwd().resolve()
CLI_PATH = REPO_ROOT / "scripts" / "filesystem_artifacts.py"

root_default = os.environ.get("ARTIFACTS_FS_ROOT", "")
index_default = os.environ.get("ARTIFACTS_FS_INDEX", "")
backend_value = os.environ.get("ARTIFACTS_BACKEND", "")

root_input = widgets.Text(value=root_default, description="FS root", layout=widgets.Layout(width="60%"))
index_input = widgets.Text(value=index_default, description="Index", layout=widgets.Layout(width="60%"))
max_bytes_input = widgets.Text(value="200g", description="Max bytes")
max_age_input = widgets.BoundedFloatText(value=14.0, min=0.0, max=365.0, step=1.0, description="Max age (days)")
min_free_input = widgets.Text(value="", description="Min free")
runs_input = widgets.Textarea(
    value="",
    placeholder="Optional run IDs (one per line)",
    description="Run filter",
    layout=widgets.Layout(width="60%", height="80px"),
)
dry_run_checkbox = widgets.Checkbox(value=True, description="Dry run only", indent=False)
assume_yes_checkbox = widgets.Checkbox(value=False, description="Auto confirm (--yes)", indent=False)
run_button = widgets.Button(description="Run prune", icon="trash")
status_label = widgets.HTML("")
log_output = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", max_height="260px", overflow="auto"))


def _format_status(message: str, *, ok: bool) -> str:
    color = "#2e7d32" if ok else "#c62828"
    return f"<span style='color:{color}; font-weight:bold'>{message}</span>"


def _build_command() -> List[str]:
    cmd = [sys.executable, str(CLI_PATH), "prune"]
    root_value = root_input.value.strip()
    index_value = index_input.value.strip()
    if root_value:
        cmd.extend(["--root", root_value])
    if index_value:
        cmd.extend(["--index", index_value])
    if max_bytes_input.value.strip():
        cmd.extend(["--max-bytes", max_bytes_input.value.strip()])
    if max_age_input.value and max_age_input.value > 0:
        cmd.extend(["--max-age-days", str(max_age_input.value)])
    if min_free_input.value.strip():
        cmd.extend(["--min-free-bytes", min_free_input.value.strip()])
    for run_id in runs_input.value.splitlines():
        run_id = run_id.strip()
        if run_id:
            cmd.extend(["--run", run_id])
    if dry_run_checkbox.value:
        cmd.append("--dry-run")
    else:
        cmd.append("--no-dry-run")
    if assume_yes_checkbox.value:
        cmd.append("--yes")
    return cmd


def _run_prune(_btn: widgets.Button) -> None:
    log_output.clear_output()
    errors = []
    effective_backend = (backend_value or os.environ.get("ARTIFACTS_BACKEND", "")).lower()
    if effective_backend != "filesystem":
        errors.append("Set ARTIFACTS_BACKEND=filesystem before pruning.")
    if not CLI_PATH.exists():
        errors.append(f"Missing CLI script: {CLI_PATH}")
    if not (max_bytes_input.value.strip() or max_age_input.value > 0 or min_free_input.value.strip()):
        errors.append("Provide at least one retention constraint (max bytes / age / min free).")
    if errors:
        status_label.value = _format_status(" ".join(errors), ok=False)
        return

    cmd = _build_command()
    env = os.environ.copy()
    pythonpath_bits = [str(REPO_ROOT), str(REPO_ROOT / "src")]
    if env.get("PYTHONPATH"):
        pythonpath_bits.append(env["PYTHONPATH"])
    env["PYTHONPATH"] = os.pathsep.join(pythonpath_bits)

    status_label.value = _format_status("Running prune command…", ok=True)

    with log_output:
        print("$", " ".join(shlex.quote(part) for part in cmd))
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env)
        assert process.stdout is not None
        for line in process.stdout:
            print(line.rstrip())
        return_code = process.wait()

    if return_code == 0:
        status_label.value = _format_status("Prune command completed.", ok=True)
    else:
        status_label.value = _format_status(f"Prune command failed (exit {return_code}).", ok=False)


run_button.on_click(_run_prune)

prune_controls = widgets.VBox(
    [
        widgets.HBox([root_input, index_input]),
        widgets.HBox([max_bytes_input, max_age_input, min_free_input]),
        runs_input,
        widgets.HBox([dry_run_checkbox, assume_yes_checkbox, run_button]),
        status_label,
        log_output,
    ]
)

display(prune_controls)


## Optional: run a stub orchestration smoke test
This cell runs the Python runner in simulation mode (fallback adapters) so you can confirm Drive folders are writable before enabling real models.

In [ ]:
# Cell 6: Optional orchestrator smoke run (uses fallback adapters)
from pathlib import Path
from sparkle_motion.orchestrator import Runner

workspace_root = DRIVE_ROOT if USE_DRIVE else LOCAL_ROOT
runs_root = workspace_root / "runs"
runs_root.mkdir(parents=True, exist_ok=True)
movie_plan = {
    "title": "Colab Smoke",
    "shots": [
        {
            "id": "shot_001",
            "visual_description": "Test scene",
            "duration_sec": 2.0,
            "dialogue": [{"character": "narrator", "text": "Hello from Colab"}],
        }
    ],
}
runner = Runner(runs_root=str(runs_root))
asset_refs = runner.run(movie_plan=movie_plan, run_id="colab_smoke", resume=True)
print("Smoke run complete. Final asset refs keys:", asset_refs.keys())
print("Runs directory:", runs_root)